<a href="https://colab.research.google.com/github/VicSkl18/-Engineering-applications/blob/main/Identifier-of-Thermodynamic-Processes/Identificador%20de%20Processos%20Termodin%C3%A2micos%20-%201.7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#===================================================================================================#
'''
Calculadora para Identificação de Processos Termodinâmicos

Versão 1.7
Autor: Victor S. Teixeira
LinkedIn: https://www.linkedin.com/in/victor-s-teixeira-022a5717b
e-mail: unieviteixeira@fei.edu.br

Sobre: 
Esse programa identifica qual processo termodinâmico ocorreu através dos dados 
de pressão e volume iniciais e finais e calcula os valores de trabalho(W), calor envolvido(Q)
e variação de energia interna(ΔEi). Além disso ele plota o diagrama PxV e permite que o usuário descubra a constante adiabática, 
permitindo que o mesmo saiba se o gás em questão é mono, poli ou diatômico.

Obs.: Essa é a atualização da versão 1.7 do programa. Agora o objivo são cálculos para sistemas gerais de medidas, encarregando o usuário a definir
qual sistema será utilizado nas entradas e quais serão os entregues no resultado final. Portanto, exige-se do estudante, professor e ou outros que
tenham pleno conhecimento de análise dimensional.
Quaisquer dúvidas, bugs ou problemas durante a execução do programa entre em contato com o desenvolvedor.
'''

#==============================================BIBLIOTECA============================================#


from numpy import log as ln 
import numpy as np
import plotly.graph_objects as go

#===========================FUNÇÃO 0: IDENTIFICAR SE O USUÁRIO TEM OU NÃO VALOR DE GAMA===========================#

def valor_gama():
    print("Obs.: A segunda opção é feita especificamente para ciclos termodinâmicos ou em casos\nonde você tem um processo adiabático para ser usado como referência. Dito isso, seus\nvalores de entrada devem ser de uma curva adiabática!")
    print("="*85)
    x = str(input('Você tem o valor de gama [S/N]? ')).strip().upper()
    
    while x not in 'SsNn':
        print("Responda apenas com 'S' par sim ou 'N' para não!!!")
        x = str(input('Você tem o valor de gama [S/N]? '))

    if x == 'S':
        n = 0
        return n

    elif x == 'N':   
        n = 1 
        return n

#===========================FUNÇÃO 1: ENTRADA DE VALORES(COM O VALOR DE GAMA)===========================#

def entrada_de_valores_com_gama():
    
#ENTRADA DE VALORES

        try:  
            print("="*85)
            g = float(input("γ = "))
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: ")) 
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)
        
        except Exception as  erro:
            print('='*60)
            print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
            entrada_de_valores_com_gama()



#Enfia os valores de entradaoara a função  "identifica_processo()"
        identifica_processo(g, pi, pf, vi, vf)

#===========================FUNÇÃO 2: IDENTIFICAÇÃO DOS PROCESSOS (COM O VALOR DE GAMA)===========================#

def identifica_processo(gama, Pi, Pf, Vi, Vf):

    #VARIÁVEIS AUXILIARES: ISOBÁRICO

            Pt = Pf - Pi
            Vt = Vf - Vi 

    #VARIÁVEIS AUXILIARES: ADIABÁICO

            P1 = Pi * (Vi ** gama)
            P1 = round(P1, 3)
            P2 = Pf * (Vf ** gama)
            P2 = round(P2, 3)

    #CONDIÇÃO PARA DIFERENCIAR OS VALORES DE CP E CV COM BASE NO TIPO DE GÁS

            if gama == 1.67 or gama == 1.4 or gama == 1.33:
                if gama == 1.67:
                    cv = (3/2)
                    cp = (5/2)

                elif gama == 1.4:
                    cv = (5/2)
                    cp = (7/2)

                elif gama == 1.33:
                    cv = 3
                    cp = 4   

     #IDENTIFICAÇÃO: ISOCÓRICO

            if Vi == Vf:

                processo = "Isocórico"

     #IDENTIFICAÇÃO: ISOBÁRICO

            elif Pi == Pf:  

                processo = "Isobárico"  
    
     #ANÁLISE: ISOTÉRMICO OU ADIABÁTICO?

            E_adi, Erro_isot = erro_percentual(P1, P2, Pi, Vi, Pf, Vf)
            
            if (Pi * Vi <= Pf * Vf) or (Pi * Vi >= Pf * Vf) or (P1 <= P2) or (P1 >= P2):

               if E_adi <= 10 or Erro_isot <= 10: 
               
                                    
                #ERRO PERCENTUAL: RESULTADOS
                  print("ADIABÁTICO: PERCENTUAL DE ERRO PiVi^γ = PfVf^γ: {:.5f}%".format(E_adi))
                  print("ISOTÉRMICO: PERCENTUAL DE ERRO Pi.Vi = Pf.Vf: {:.5f}%".format(Erro_isot))
                  print()
                  print("Com  base nesses dados qual é o processo?")
                  print()
                  resp1 = isotermico_ou_adiabatico() 

                  if resp1 == '1':

                    #IDENTIFICAÇÃO: ADIABÁTICO

                        processo = "Adiabático"
                 
                  if resp1 == '2':             
                                
                    #IDENTIFICAÇÃO: ISOTÉRMICO

                        processo = "Isotérmico"
              
            
     
     #Envia os dados para a função do respectivo processo

            if processo == "Isocórico":

               isocorico(Pi, Pf, Vi, Vf, cv)

            
            elif processo == "Isobárico":

                isobárico(Pi, Pf, Vi, Vf, cv, cp, Pt, Vt)

            elif processo == "Isotérmico":

                isotermico(Pi, Pf, Vi, Vf)    
            
            elif processo == "Adiabático":

                adiabatico(gama, Pi, Pf, Vi, Vf)

#===========================FUNÇÃO 3: CALCULA O PERCENTUAL DE ERRO PARA ISOTÉRMICO E ADIABÁTICO===========================#

def erro_percentual(p1, p2, PI, VI, PF, VF):

    #PERCENUAL DE ERRO: ADIABÁTICO
         
    if p1 >  p2:

        maior1 = p1 
        menor1 = p2
    
    elif p1 < p2:

        maior1 = p2  
        menor1 = p1  

    else:

        maior1 = p1
        menor1 = p2     

    Erro1 = (100 *((maior1 - menor1))) / maior1

    #PERCENTUAL DE ERRO: ISOTÉRMICO
    
    PV1 = PI * VI
    PV2 = PF * VF
    
    if  PV1 > PV2:

        maior2 = PV1
        menor2 = PV2
    
    elif  PV1 < PV2:

        maior2 = PV2
        menor2 = PV1
   
    else:

        maior2 = PV1
        menor2 = PV2

    Erro2 = (100 *((maior2 - menor2))) / maior2

    return Erro1, Erro2            

#===========================FUNÇÃO 5: VERIFICA SE O USUÁRIO SELECIONOU O ISOTÉRMICO OU ADIABÁTICO===========================#
def isotermico_ou_adiabatico():
    
    print('''
[1] Adiabático
[2] Isotérmico
            ''') 
    print('='*60)

    RESP = str(input("Qual opção deseja? "))

    print('='*60)

    while RESP not in ['1', '2']: 

        print("INSIRA APENAS '1' OU '2' !!!")
        RESP = input("Qual opção deseja? ")
        print('='*60)

    return RESP             

#===========================FUNÇÃO 6: PROCESSO ISOCÓRICO===========================#
def isocorico(Pi1, Pf1, Vi1, Vf1, cv1):

    #Cálculos: Trabalho, calor envolvido, variação de energia interna

    W = 0
    Q = cv1 * (Pf1 - Pi1) * Vf1
    Ei = Q

    #Resultado dos cálculos e nome do processo

    print("Processo termodinâmico: Isocórico") 
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print('='*60)

    #Diagrama PxV

    dd_x = [Vi1, Vf1]
    dd_y = [Pi1, Pf1]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dd_x, y = dd_y, mode = 'lines')) 
    fig.update_layout(title='Diagrama p.V: Processo Isocórico',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

#VERIFICA SE O USUÁRIO QUER OU NÃO CONTINUAR USANDO A CALCULADORA

    calcular_novamente()

#===========================FUNÇÃO 6: PROCESSO ISOBÁRICO===========================#
def isobárico(Pi1, Pf1, Vi1, Vf1, cv1, cp1, Pt1, Vt1):

    #Cálculos: Trabalho, calor envolvido, variação de energia interna

    Pt1 = Pf1
    W = Pt1 * Vt1
    Q = cp1 * W
    Ei = cv1 * Pt1 * Vt1

    #Resultado dos cálculos e nome do processo

    print("Processo termodinâmico: Isobárico") 
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print('='*60)

    #Diagrama PxV

    dd_x = [Vi1, Vf1]
    dd_y = [Pi1, Pf1]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dd_x, y = dd_y, mode = 'lines')) 
    fig.update_layout(title='Diagrama p.V: Processo Isobárico',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

#VERIFICA SE O USUÁRIO QUER OU NÃO CONTINUAR USANDO A CALCULADORA

    calcular_novamente()

#===========================FUNÇÃO 7: PROCESSO ISOTÉRMICO===========================#
def isotermico(Pi1, Pf1, Vi1, Vf1):

    #Cálculos: Trabalho, calor envolvido, variação de energia interna

    W = Pf1 * Vf1 * ln((Vf1 / Vi1))
    Q = W
    Ei = 0

    #Resultado dos cálculos e nome do processo

    print("Processo termodinâmico: Isotérmico") 
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print('='*60)

    #Diagrama PxV
    k = Pi1 * Vi1
   
    if Pi1 > Pf1:
      pr = np.arange(Pi1, Pf1, (Pf1-Pi1) / 20)

    else:
      pr = np.arange(Pf1, Pi1, (Pi1-Pf1) / 20)

    vl = k / pr

    fig = go.Figure()
    fig.add_trace(go.Scatter(x = vl_x, y = pr, mode = 'lines')) 
    fig.update_layout(title='Diagrama p.V: Processo Isotérmico',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

#VERIFICA SE O USUÁRIO QUER OU NÃO CONTINUAR USANDO A CALCULADORA

    calcular_novamente()

#===========================FUNÇÃO 8: PROCESSO ADIABÁTICO===========================#
def adiabatico(gama1, Pi1, Pf1, Vi1, Vf1):

    #Cálculos: Trabalho, calor envolvido, variação de energia interna

    W = (((Pi1 * Vi1) - (Pf1 * Vf1)) / (gama1 - 1))
    Q = 0
    Ei = -W
    #Resultado dos cálculos e nome do processo

    print("Processo termodinâmico: Adiabático") 
    print("W = {:.3f}".format(W))
    print("Q = {:.3f}".format(Q))
    print("ΔEi = {:.3f}".format(Ei))
    print("="*85)

    #Diagrama PxV
    k = Pi1 * (Vi1 ** gama1)

    if Pi1 > Pf1:
      pr = np.arange(Pi1, Pf1, (Pf1-Pi1) / 20)

    else:
      pr = np.arange(Pf1, Pi1, (Pi1-Pf1) / 20)
    
    vl = (k /pr) ** (1/gama1)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x = vl_x, y = pr, mode = 'lines')) 
    fig.update_layout(title='Diagrama p.V: Processo Adiabático',
    xaxis_title='Volume',
    yaxis_title='Pressão',
    height = 400, width = 600,
    plot_bgcolor = 'white',
    font = {'family': 'Arial','size': 16,'color': 'black'})
    fig.update_xaxes( showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tick0 = 0,  dtick = 1, exponentformat = "power",
    showline=True, linewidth=1, linecolor='black')
    fig.show()

#VERIFICA SE O USUÁRIO QUER OU NÃO CONTINUAR USANDO A CALCULADORA

    calcular_novamente()

#===========================FUNÇÃO 9:ENTRADA DE VALORES SEM GAMA===========================#

def entrada_de_valores_sem_gama():   
 
    try:  

        print("="*85)
        pi   = float(input("Insira o valor da pressão inicial: "))
        pf   = float(input("Insira o valor da pressão final: "))
        vi   = float(input("Insira o valor do volume inicial: ")) 
        vf   = float(input("Insira o valor do volume final: "))
        print("="*85)
    
    except Exception as  erro:
        print("="*85)
        print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
        entrada_de_valores_sem_gama()



#Enfia os valores de entradaoara a função  "identifica_processo()"
    descobre_gama(pi, pf, vi, vf)

#===========================FUNÇÃO 10: DESCOBRE O VALOR DE GAMA========================#
def descobre_gama(pi1, pf1, vi1, vf1):

  P = (pi1/pf1)
  V = (vf1/vi1)

  mono = 1.67 * 0.05
  dia = 1.4 * 0.05
  poli = 1.33 * 0.05

  gama = ln(P) / ln(V)

  if gama >= (1.67  * (1 - mono)) and gama <= (1.67 * (1 + mono)) or  gama == 1.67:

      print("γ = {}".format(1.67))
      print("Você está trabalhando com um gás monoatômico!")
      print('='*60)
      r = calcular_com_gama()

      if r == 0:
        entrada_de_valores_com_gama2(1.67)

      elif r == 1:
    
        print("CALCULADORA ENCERRADA!!!")
        print("="*85)
        

  elif gama >= (1.4  * (1 - dia)) and gama <= (1.4 * (1 + dia)) or  gama == 1.4:

      print("γ = {}".format(1.4)) 
      print("Você está trabalhando com um gás diatômico!")
      print('='*60)
      r = calcular_com_gama()

      if r == 0:
        entrada_de_valores_com_gama2(1.4)


      elif r == 1:
    
        print("CALCULADORA ENCERRADA!!!")
        print("="*85)
        

  elif gama >= (1.33  * (1 - poli)) and gama <= (1.33 * (1 + poli)) or gama == 1.33:

      print("γ = {}".format(1.33)) 
      print("Você está trabalhando com um gás politômico!")
      print("="*85)
      r = calcular_com_gama()

      if r == 0:
        entrada_de_valores_com_gama2(1.33)

      
      elif r == 1:
    
        print("CALCULADORA ENCERRADA!!!")
        print("="*85)

  else:

      print("Os valores de entrada não condizem com um processo adiabático\nportanto não foi possível descobrir o valor de gama!")
      print("Insira novos valores que atendam o requisitado.")
      entrada_de_valores_sem_gama()

#===========================FUNÇÃO 11: VERIFICA SE O USUÁRIO QUER CALCULAR OUTRO PROCESSO COM 'γ'===========================#
def calcular_com_gama():
    x = str(input('Você deseja utilizar "γ" para calcular outro processo? [S/N] ')).strip().upper()
    
    while x not in 'SsNn':
        print("Responda apenas com 'S' par sim ou 'N' para não!!!")
        x = str(input('Você deseja utilizar "γ" para calcular outro processo? [S/N] '))

    if x == 'S':
        n = 0
        return n

    elif x == 'N':   
        n = 1 
        return n
#===========================FUNÇÃO 12: ENTRADA DE VALORES(COM O VALOR DE GAMA) 2===========================#

def entrada_de_valores_com_gama2(g1):
    
#ENTRADA DE VALORES
        g = g1

        try:  
            print("="*85)          
            pi   = float(input("Insira o valor da pressão inicial: "))
            pf   = float(input("Insira o valor da pressão final: "))
            vi   = float(input("Insira o valor do volume inicial: ")) 
            vf   = float(input("Insira o valor do volume final: "))
            print("="*85)
        
        except Exception as  erro:
            print("="*85)
            print("Um dos valores inseridos não é um número,\nnão será possível realizar os cálculos.\nInsira os valores novamente!")
            entrada_de_valores_com_gama()



#Enfia os valores de entradaoara a função  "identifica_processo()"
        identifica_processo(g, pi, pf, vi, vf)
 
#===========================FUNÇÃO 13: VERIFICA SE O USUÁRIO QUER CONINUAR CALCULANDO===========================#

def calcular_novamente():

   print('''
[1] Fazer outro cálculo
[2] Encerrar calculadora
            ''') 
   print("="*85)

   RESP = str(input("Qual opção deseja? "))

   while RESP not in ['1', '2']: 

        print("INSIRA APENAS '1' OU '2' !!!")
        RESP = input("Qual opção deseja? ")
        
   if RESP == '1':
     print("="*85)
     vg = valor_gama()

     if vg == 0:
      entrada_de_valores_com_gama()

     if vg == 1:
       entrada_de_valores_sem_gama() 


   elif RESP == '2':
      print("="*85)
      print("CALCULADORA ENCERRADA!!!")
      print("="*85)
      


#===========================PROGRAMA PRINCIPAL: INFORMAÇÕES===========================#

print("="*85)
print("\033[1m CÁLCULO E IDENTIFICAÇÃO DE PROCESSOS TERMODINÂMICOS \033[0m".center(85))
print("="*85)
print("\033[1;31mA escolha do sistema de unidades é exclusiva do usuário portanto, cabe ao usuário\nter pleno conhecimento de análise dimensional para uma boa interpretação do gráfico e\ndos resultados.\033[0m")

print("="*85)
print("Constante γ:")
print("Monoatômico = 1.67\nDiatômico   = 1.4\nPoliatômico = 1.33")
print("="*85)

vg = valor_gama()

if vg == 0:
  entrada_de_valores_com_gama()
   
if vg == 1:
  entrada_de_valores_sem_gama()   

             CÁLCULO E IDENTIFICAÇÃO DE PROCESSOS TERMODINÂMICOS             
A escolha do sistema de unidades é exclusiva do usuário portanto, cabe ao usuário
ter pleno conhecimento de análise dimensional para uma boa interpretação do gráfico e
dos resultados.
Constante γ:
Monoatômico = 1.67
Diatômico   = 1.4
Poliatômico = 1.33
Obs.: A segunda opção é feita especificamente para ciclos termodinâmicos ou em casos
onde você tem um processo adiabático para ser usado como referência. Dito isso, seus
valores de entrada devem ser de uma curva adiabática!
Você tem o valor de gama [S/N]? s
γ = 1.67
Insira o valor da pressão inicial: 2
Insira o valor da pressão final: 2
Insira o valor do volume inicial: 2
Insira o valor do volume final: 6
Processo termodinâmico: Isobárico
W = 8.000
Q = 20.000
ΔEi = 12.000



[1] Fazer outro cálculo
[2] Encerrar calculadora
            
Qual opção deseja? 2
CALCULADORA ENCERRADA!!!
